In [2]:
import os
import io
import sys
import re
import time
import math
import pickle

import numpy as np
import pandas as pd

from numpy import nan as Nan
from numpy import inf as inf
from tqdm import tqdm
from scipy.sparse import csr_matrix
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
import matplotlib.pyplot as plt
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.ensemble import GradientBoostingClassifier
from catboost import CatBoostRegressor, Pool, EShapCalcType, EFeaturesSelectionAlgorithm
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from data_preprocessing import *
from sklearn.model_selection import KFold
from utils import *
from scoring import * 

In [10]:
values = [1, 2, 3]

with open("file.txt", "w") as output:
    output.write(str(values))

In [12]:
import json

with open("file.txt", "r") as f:
    array = json.load(f)

In [15]:
array

[1, 2, 3]

In [3]:
dfs = []
for i in range(5):
    dfs.append(load_csv(f"groups_and_oxi_states_5_frames/df_features_with_barrier_step_{i}.pkl"))

In [4]:
dfs[0]['stru_label'].to_list() == dfs[1]['stru_label'].to_list() == dfs[2]['stru_label'].to_list() == dfs[3]['stru_label'].to_list() == dfs[4]['stru_label'].to_list()

True

In [4]:
preds_without_traj, y = test_function_without_trajectories(dfs[0], number_of_folds = 3, random_state = 0)
bootstrap_roc_auc(1000, y, preds_without_traj)

TypeError: 'module' object is not callable

In [15]:
preds_with_traj, preds_without_traj, y = test_function_with_trajectories(dfs, number_of_folds = 3, random_state = 0)

roc auc score for train: 1.0, for test 0.8777777777777778
roc auc score for train: 1.0, for test 0.8814814814814814
roc auc score for train: 1.0, for test 0.9666666666666667
roc auc score for train: 1.0, for test 0.9370370370370371
roc auc score for train: 1.0, for test 0.9962962962962963
roc auc score for train: 1.0, for test 0.9184782608695652
roc auc score for train: 1.0, for test 0.9755434782608696
roc auc score for train: 1.0, for test 0.9429347826086956
roc auc score for train: 1.0, for test 0.9320652173913043
roc auc score for train: 1.0, for test 0.9510869565217391
roc auc score for train: 1.0, for test 0.8095238095238095
roc auc score for train: 1.0, for test 0.8235294117647058
roc auc score for train: 0.989811320754717, for test 0.8515406162464986
roc auc score for train: 0.9984905660377359, for test 0.8403361344537815
roc auc score for train: 1.0, for test 0.8207282913165266


In [17]:
bootstrap_roc_auc(1000, y, preds_with_traj)

(0.9136169346361901, 0.026919651239747162)

In [18]:
bootstrap_roc_auc(1000, y, preds_without_traj)

(0.8352166142789906, 0.040838305791136356)

In [ ]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa

In [ ]:
splits = [8334788, 9151291, 4517925, 5765657, 1797675]
roc_auc = []
for random_split in splits:
    assembled_pred = np.zeros(47)
    for i, df in enumerate(dfs):
        X, y = (df.drop(['is_good', 'stru_label', 'stru_id'], axis=1), df['is_good'])
        X = X.to_numpy()
        y = y.astype(int)
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        if i == 0:
            X_tr, X_te, y_tr, y_te = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
        else:
            X_tr, _, y_tr, _ = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
        model = CatBoostClassifier(eval_metric='AUC', verbose = False)
        feature_names = ['F{}'.format(i) for i in range(np.array(X_tr).shape[1])]
        test_pool = Pool(np.array(X_te), y_te, feature_names=feature_names)
        summary = model.select_features(
        X = X_tr,
        y=y_tr,
        eval_set=test_pool,
        features_for_select= np.arange(len(X_tr[0])),
        num_features_to_select=50,
        steps=6,
        algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
        shap_calc_type=EShapCalcType.Regular,
        train_final_model=True,
        logging_level='Silent',
        plot=False)
        y_pred = model.predict_proba(X_te)[:, 1]
        assembled_pred += y_pred
        print(f"roc auc score {roc_auc_score(y_te, y_pred)}")
        indexes = np.argsort(model.predict_proba(X_te)[:, 1])
    assembled_pred /= len(dfs)
    roc_auc.append(roc_auc_score(y_te, assembled_pred))
    print(f"roc auc score {roc_auc[-1]} for random_split {random_split}")
print(f"mean roc auc {np.mean(roc_auc)}")

roc auc score 0.9248120300751879
roc auc score 0.9154135338345865
roc auc score 0.9379699248120301
roc auc score 0.8383458646616541
roc auc score 0.8853383458646618
roc auc score 0.9022556390977443 for random_split 8334788
roc auc score 0.9133064516129032
roc auc score 0.8326612903225806
roc auc score 0.8366935483870968
roc auc score 0.9092741935483871
roc auc score 0.8588709677419355
roc auc score 0.8850806451612904 for random_split 9151291
roc auc score 0.8113553113553112
roc auc score 0.912087912087912
roc auc score 0.8406593406593407
roc auc score 0.8498168498168498
roc auc score 0.8901098901098901
roc auc score 0.8736263736263736 for random_split 4517925
roc auc score 0.8509803921568627
roc auc score 0.8725490196078433
roc auc score 0.888235294117647
roc auc score 0.8686274509803922
roc auc score 0.9235294117647059
roc auc score 0.903921568627451 for random_split 5765657
roc auc score 0.9745098039215686
roc auc score 0.9431372549019609
roc auc score 0.9372549019607843
roc auc scor

In [ ]:
splits = [8334788, 9151291, 4517925, 5765657, 1797675]
roc_auc = []
for random_split in splits:
    assembled_pred = np.zeros(46)
    for df in dfs:
        X, y = (df.drop(['is_good', 'stru_label'], axis=1), df['is_good'])
        X_scaled = scaler.fit_transform(X)
        y = y.astype(int)
        X_tr, X_te, y_tr, y_te = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
        model = CatBoostClassifier(eval_metric='AUC', verbose = False)
        feature_names = ['F{}'.format(i) for i in range(np.array(X_tr).shape[1])]
        test_pool = Pool(np.array(X_te), y_te, feature_names=feature_names)
        summary = model.select_features(
        X = X_tr,
        y=y_tr,
        eval_set=test_pool,
        features_for_select= np.arange(len(X_tr[0])),
        num_features_to_select=20,
        steps=6,
        algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
        shap_calc_type=EShapCalcType.Regular,
        train_final_model=True,
        logging_level='Silent',
        plot=False)
        y_pred = model.predict_proba(X_te)[:, 1]
        assembled_pred += y_pred
        print(f"roc auc score {roc_auc_score(y_te, y_pred)}")
        indexes = np.argsort(model.predict_proba(X_te)[:, 1])
    assembled_pred /= len(dfs)
    roc_auc.append(roc_auc_score(y_te, assembled_pred))
    print(f"roc auc score {roc_auc[-1]} for random_split {random_split}")
print(f"mean roc auc {np.mean(roc_auc)}")

roc auc score 0.8825757575757576
roc auc score 0.9015151515151515
roc auc score 0.9166666666666666
roc auc score 0.9431818181818181
roc auc score 0.9071969696969697
roc auc score 0.9318181818181819 for random_split 8334788
roc auc score 0.8038461538461539
roc auc score 0.823076923076923
roc auc score 0.9
roc auc score 0.8576923076923078
roc auc score 0.8730769230769231
roc auc score 0.8692307692307693 for random_split 9151291
roc auc score 0.9412878787878788
roc auc score 0.928030303030303
roc auc score 0.8674242424242424
roc auc score 0.8920454545454546
roc auc score 0.8920454545454546
roc auc score 0.9261363636363635 for random_split 4517925
roc auc score 0.9288461538461539
roc auc score 0.6673076923076924
roc auc score 0.8173076923076923
roc auc score 0.8769230769230769
roc auc score 0.8884615384615385
roc auc score 0.823076923076923 for random_split 5765657
roc auc score 0.9142300194931774
roc auc score 0.949317738791423
roc auc score 0.9083820662768031
roc auc score 0.951267056530

In [ ]:
splits = [8334788, 9151291, 4517925, 5765657, 1797675]
roc_auc = []
for random_split in splits:
    assembled_pred = np.zeros(46)
    for df in dfs:
        grand_X = list()
        grand_y = list()
        X, y = (df.drop(['is_good', 'stru_label'], axis=1), df['is_good'])
        X_scaled = scaler.fit_transform(X)
        grand_X.append(X_scaled)
        y = y.astype(int)
        grand_y.append(y)
        X_tr = list()
        y_tr = list()
        X_te = list()
        y_te = list()
        for X_scaled, y in zip(grand_X, grand_y):
            X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
            X_tr.extend(X_train)
            X_te.extend(X_test)
            y_tr.extend(y_train)
            y_te.extend(y_test)
        model = CatBoostClassifier(eval_metric='AUC', verbose = False)
        feature_names = ['F{}'.format(i) for i in range(np.array(X_tr).shape[1])]
        test_pool = Pool(np.array(X_te), y_te, feature_names=feature_names)
        summary = model.select_features(
        X = X_tr,
        y=y_tr,
        eval_set=test_pool,
        features_for_select= np.arange(len(X_tr[0])),
        num_features_to_select=50,
        steps=3,
        algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
        shap_calc_type=EShapCalcType.Regular,
        train_final_model=True,
        logging_level='Silent',
        plot=False)
        y_pred = model.predict_proba(X_te)[:, 1]
        assembled_pred += y_pred
        print(f"roc auc score {roc_auc_score(y_te, y_pred)}")
        indexes = np.argsort(model.predict_proba(X_te)[:, 1])
    assembled_pred /= len(dfs)
    roc_auc.append(roc_auc_score(y_te, assembled_pred))
    print(f"roc auc score {roc_auc[-1]} for random_split {random_split}")
print(f"mean roc auc {np.mean(roc_auc)}")

roc auc score 0.9034090909090908
roc auc score 0.8939393939393939
roc auc score 0.9034090909090908
roc auc score 0.9109848484848486
roc auc score 0.8996212121212122
roc auc score 0.9242424242424242 for random_split 8334788
roc auc score 0.8076923076923077
roc auc score 0.8153846153846154
roc auc score 0.8461538461538461
roc auc score 0.8576923076923078
roc auc score 0.6749999999999999
roc auc score 0.8403846153846154 for random_split 9151291
roc auc score 0.8570075757575758
roc auc score 0.9166666666666666
roc auc score 0.7443181818181818
roc auc score 0.9034090909090909
roc auc score 0.8257575757575757
roc auc score 0.9090909090909092 for random_split 4517925
roc auc score 0.8730769230769231
roc auc score 0.8865384615384615
roc auc score 0.675
roc auc score 0.6557692307692308
roc auc score 0.8807692307692307
roc auc score 0.8365384615384616 for random_split 5765657
roc auc score 0.8304093567251463
roc auc score 0.9512670565302143
roc auc score 0.9473684210526315
roc auc score 0.945419

In [ ]:
def feature_preprocessing(dfs, include_trajectories = False):
    grand_X = list()
    grand_y = list()
    for df in dfs:
        scaler = StandardScaler()
        X, y = (df.drop(['is_good', 'stru_label'], axis=1), df['is_good'])
        X_scaled = scaler.fit_transform(X)
        grand_X.append(X_scaled)
        y = y.astype(int)
        grand_y.append(y)
        if not include_trajectories:
            break
    return grand_X, grand_y

def get_train_test(grand_X, grand_y, random_split):
    X_tr = list()
    y_tr = list()
    X_te = list()
    y_te = list()
    for X_scaled, y in zip(grand_X, grand_y):
        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
        X_tr.extend(X_train)
        X_te.extend(X_test)
        y_tr.extend(y_train)
        y_te.extend(y_test)
    X_tr = np.array(X_tr)
    X_te = np.array(X_te)
    return X_tr, X_te, y_tr, y_te

In [ ]:
def hyperparameters_of_select_features(num_features, steps):
    grand_X, grand_y = feature_preprocessing(dfs, True)

    roc_auc = []
    splits = [8334788, 9151291, 4517925, 5765657, 1797675]


    for random_split in splits:
        assembled_pred = np.zeros(46)
        for X_scaled, y in zip(grand_X, grand_y):
            X_te = list()
            y_te = list()
            X_tr, X_te, y_tr, y_te = train_test_split(X_scaled, y, test_size=0.4, random_state=random_split)
            model = CatBoostClassifier(eval_metric='AUC', verbose = False)
            feature_names = ['F{}'.format(i) for i in range(np.array(X_tr).shape[1])]
            test_pool = Pool(np.array(X_te), y_te, feature_names=feature_names)
            summary = model.select_features(
            X = X_tr,
            y=y_tr,
            eval_set=test_pool,
            features_for_select= np.arange(len(X_tr[0])),
            num_features_to_select=num_features,
            steps=steps,
            algorithm=EFeaturesSelectionAlgorithm.RecursiveByShapValues,
            shap_calc_type=EShapCalcType.Regular,
            train_final_model=True,
            logging_level='Silent',
            plot=False)
            y_pred = model.predict_proba(X_te)[:, 1]
            print(roc_auc_score(y_te, y_pred), random_split)
            assembled_pred += y_pred
        assembled_pred /= len(dfs)
        print(f"roc auc score {roc_auc_score(y_te, assembled_pred)} for random split {random_split}")
        roc_auc.append(roc_auc_score(y_te, assembled_pred))
    print(f"mean roc auc score {np.mean(roc_auc)}")

num_features_to_select=80, steps=6 for all trajectories

667 - 577 = 90